kaggle 데이터셋 : https://www.kaggle.com/c/nlp-getting-started/overview 

In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('./kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./kaggle/input\nlp-getting-started\sample_submission.csv
./kaggle/input\nlp-getting-started\test.csv
./kaggle/input\nlp-getting-started\train.csv


In [2]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
train = pd.read_csv('./kaggle/input/nlp-getting-started/train.csv')
test = pd.read_csv('./kaggle/input/nlp-getting-started/test.csv')
submission = pd.read_csv('./kaggle/input/nlp-getting-started/sample_submission.csv')

In [4]:
train.shape, test.shape, submission.shape

((7613, 5), (3263, 4), (3263, 2))

In [5]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
submission.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [8]:
labels = train.target
labels

0       1
1       1
2       1
3       1
4       1
       ..
7608    1
7609    1
7610    1
7611    1
7612    1
Name: target, Length: 7613, dtype: int64

In [9]:
sentences = train.text
sentences

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object

In [48]:
# import nltk
# nltk.__version__
# nltk.download()

#### 불용어 제거 부분 수정 필요!!

In [50]:
# 불용어 제거
from nltk.corpus import stopwords
sw = stopwords.words('english')
np.array(sw)

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\sjbang/nltk_data'
    - 'c:\\Users\\sjbang\\anaconda3\\envs\\tf\\nltk_data'
    - 'c:\\Users\\sjbang\\anaconda3\\envs\\tf\\share\\nltk_data'
    - 'c:\\Users\\sjbang\\anaconda3\\envs\\tf\\lib\\nltk_data'
    - 'C:\\Users\\sjbang\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [17]:
def stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    return " ".join(text)
sentences = sentences.apply(stopwords)
sentences.head(10)

NameError: name 'sw' is not defined

### Train과 Val 나누기

In [32]:
from sklearn.model_selection import train_test_split
train_sentences, valid_sentences, train_labels, valid_labels = train_test_split(sentences, labels, test_size=0.2, random_state=2020)

In [33]:
train_sentences.shape, valid_sentences.shape, train_labels.shape, valid_labels.shape

((6090,), (1523,), (6090,), (1523,))

### 토큰화

In [34]:
vocab_size = 1000
token = Tokenizer(num_words = vocab_size)
token.fit_on_texts(sentences)

In [37]:
print(train_sentences[:5])
train_sequences = token.texts_to_sequences(train_sentences)
valid_sequences = token.texts_to_sequences(valid_sentences)
print(train_sentences[:5])

6477    It still hasn't sunk in that I've actually met...
2524    In times of desolation and trouble Daniel's pe...
439        @Casper_rmg @BestComedyVine whats cracking cuz
1824                @olliebailey11 havnt you crashed ? ??
4386    Remove the http://t.co/7IEiZ619h0 and Linkury ...
Name: text, dtype: object
6477    It still hasn't sunk in that I've actually met...
2524    In times of desolation and trouble Daniel's pe...
439        @Casper_rmg @BestComedyVine whats cracking cuz
1824                @olliebailey11 havnt you crashed ? ??
4386    Remove the http://t.co/7IEiZ619h0 and Linkury ...
Name: text, dtype: object


### 패딩

In [38]:
trunc_type = 'post'
padding_type = 'post'
max_length = 120
train_pad = pad_sequences(train_sequences, truncating=trunc_type, padding=padding_type, maxlen=max_length)
valid_pad = pad_sequences(valid_sequences, truncating=trunc_type, padding=padding_type, maxlen=max_length)

### 모델

In [39]:
train_labels = np.asarray(train_labels)
valid_labels = np.asarray(valid_labels)

In [40]:
embedding_dim = 64
model = tf.keras.Sequential([
    Embedding(vocab_size, embedding_dim),
    Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    Bidirectional(tf.keras.layers.LSTM(32)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [41]:
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [42]:
model.fit(train_pad, train_labels, validation_data=(valid_pad, valid_labels), epochs=3, verbose=2)

Epoch 1/3
191/191 - 17s - 89ms/step - accuracy: 0.7141 - loss: 0.5565 - val_accuracy: 0.7991 - val_loss: 0.4508
Epoch 2/3
191/191 - 11s - 58ms/step - accuracy: 0.8154 - loss: 0.4273 - val_accuracy: 0.8050 - val_loss: 0.4513
Epoch 3/3
191/191 - 11s - 59ms/step - accuracy: 0.8235 - loss: 0.3976 - val_accuracy: 0.7925 - val_loss: 0.4621


### 데이터 전처리 및 예측

In [44]:
test_sequences = test.text
test_sequences = test_sequences.apply(stopwords)
test_sequences = token.texts_to_sequences(test_sequences)
test_pad = pad_sequences(test_sequences, truncating=trunc_type, padding=padding_type, maxlen=max_length)

NameError: name 'sw' is not defined

In [45]:
y_pred=model.predict(test_pad)

NameError: name 'test_pad' is not defined

In [46]:
sub_file=pd.DataFrame()
sub_file['id']=test['id']
sub_file['target']=y_pred.round().astype(int)
sub_file.head()

NameError: name 'y_pred' is not defined

In [47]:
sub_file.to_csv('submission_lstm.csv', index=False)